<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/langchain_MISTRAL_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/v0.1/docs/integrations/chat/mistralai/

In [ ]:
!pip install colab-env --quiet
!pip install mistralai --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet
from datasets import load_dataset

In [ ]:
!pip install -U langchain-core langchain-mistralai -q

In [ ]:
import mistralai
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
import colab_env
import json

api_key = os.environ["MISTRAL_API_KEY"]
client = MistralClient(api_key=api_key)

In [4]:
from langchain_core.messages import HumanMessage
from langchain_mistralai.chat_models import ChatMistralAI

# If api_key is not passed, default behavior is to use the `MISTRAL_API_KEY` environment variable.
llm = ChatMistralAI(api_key=api_key,model_name='open-mixtral-8x7b')

messages = [HumanMessage(content="knock knock")]
response=llm.invoke(messages)
print(response.content)

Who's there?

Caramel:
Caramel who?

Caramel:
Caramel, can I come in and have some coffee with you?


In [ ]:
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

In [ ]:
!pip install langchain_community -q

In [ ]:
messages = [HumanMessage(content="the capital of canada?")]
response = llm(messages)

In [8]:
print(response.content)

The capital of Canada is Ottawa. It is located in eastern Ontario, near the city of Montreal and the U.S. border. Ottawa is home to many national institutions, including the Parliament of Canada, the Supreme Court of Canada, and a number of museums and galleries. It is also a important technology and research hub, with a number of universities and research institutions located in the city. Ottawa is known for its picturesque setting on the Ottawa River, and its many parks and green spaces make it a popular destination for outdoor enthusiasts.


In [ ]:
!pip install openai -q

In [ ]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
%cd /content/
!wget "https://www.dropbox.com/s/f6bmb19xdg0xedm/paul_graham_essay.txt?dl=1" -O pg_essay.txt
#!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt' -O pg_essay.txt

In [ ]:
!pip install langchain -q
#!pip install langchain-core -q
!pip install langchain-community -q
!pip install langchain-anthropic -q

In [12]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/content/pg_essay.txt')
data = loader.load()

In [13]:
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = splitter.split_documents(data)

In [14]:
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00


In [15]:
!pip install faiss-gpu -q
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.2 MB/s eta 0:00:00


In [ ]:
#from langchain_community.embeddings import CohereEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
#embeddings = OpenAIEmbeddings(model='text-embedding-3-small')
vector = FAISS.from_documents(documents, embeddings)

In [17]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
retriever = vector.as_retriever()

# create a chain to answer questions
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [20]:
def chat_with_ai(prompt):
  formatted_prompt = f"Instruct: Answer the following question.\n{prompt}\n"
  result = qa({"query": formatted_prompt})

  print()
  #print('chain to answer questions')
  print("-" * 80)
  print()
  result = qa({"query": prompt})
  print(f'Query: {result["query"]}\n')
  print(f'Result: {result["result"]}\n')
  print(f'Context Documents: ')
  for srcdoc in result["source_documents"]:
        print(f'{srcdoc}\n')
  print("-" * 80)

In [23]:
prompt="What did the author do growing up?"
chat_with_ai(prompt)


--------------------------------------------------------------------------------

Query: What did the author do growing up?

Result: The author, Paul Graham, worked on writing short stories and programming during his childhood, before college. He started writing stories as exercises for beginning writers, and attempted to write his first programs on an IBM 1401 in the basement of his junior high school. Later in life, he continued to write essays and also worked on spam filters, painting, cooking, and real estate. He also started publishing essays online, which he considers a turning point in his career.

Context Documents: 
page_content='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I 

In [22]:
prompt="who is the president of russia?"
chat_with_ai(prompt)


--------------------------------------------------------------------------------

Query: who is the president of russia?

Result: I'm sorry for any confusion, but the context you've provided doesn't contain any information about the current president of Russia. As of my last update, the president of Russia is Vladimir Putin. However, I recommend checking the most recent sources to confirm, as this information can change.

Context Documents: 
page_content="When we asked Sam if he wanted to be president of YC, initially he said no. He wanted to start a startup to make nuclear reactors. But I kept at it, and in October 2013 he finally agreed. We decided he'd take over starting with the winter 2014 batch. For the rest of 2013 I left running YC more and more to Sam, partly so he could learn the job, and partly because I was focused on my mother, whose cancer had returned.\n\nShe died on January 15, 2014. We knew this was coming, but it was still hard when it did.\n\nI kept working on YC ti